In [1]:
import main.Constants as Constants
from  openai import OpenAI
import os
import numpy as np
import pandas as pd
import time

import gspread
import gspread_dataframe as gd
import gspread_formatting as gf
from gspread_formatting import cellFormat, color, textFormat

## Downloading Quiz Score

In [3]:
# No need to provide path for service account.  Json file is in the default directory for gspread at %APPDATA%/gspread/service_account.json
sa = gspread.service_account()
sh = sa.open("New Chinese Words")

sheet_name = "QuizLog"
wks2 = sh.worksheet(sheet_name)
print(f'Sheet {sheet_name} Num Rows: {wks2.row_count}')
print(f'Sheet {sheet_name} Num Columns: {wks2.col_count}')


Sheet QuizLog Num Rows: 989
Sheet QuizLog Num Columns: 24


In [15]:
quiz_history = pd.DataFrame(wks2.get_all_values())
quiz_history.columns = quiz_history.iloc[0]
quiz_history = quiz_history.iloc[1:]
quiz_history.head()

for col in quiz_history.columns:
    quiz_history[col] = pd.to_numeric(quiz_history[col], errors='ignore')   

C:\Users\wongs\AppData\Local\Temp\ipykernel_1452\3291478138.py:7: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  quiz_history[col] = pd.to_numeric(quiz_history[col], errors='ignore')


In [22]:
quiz_history['Pinyin Wrong'] = (quiz_history['Pinyin Correction']!='').astype(int)
quiz_history['Meaning Wrong'] = (quiz_history['Meaning Correction']!='').astype(int)

In [25]:
quiz_history.loc[quiz_history['Word']=='麻烦']

,Word,Pinyin Answer,Pinyin Correct,Pinyin Correction,Meaning,Meaning Correct,Meaning Correction,Right Score,Wrong Score,Last Quiz,Quiz Id,Pinyin Wrong,Meaning Wrong
122,麻烦,ma2fan3,no,máfan,troublesome,yes,,0,1,2024-12-29,QW000010,1,0
458,麻烦,ma2fan3,no,ma2fan,trouble,yes,,0,1,2025-01-29,QW000017,1,0
546,麻烦,ma2fan2,no,ma2fan,trouble,yes,,0,1,2025-06-21,QW000027,1,0


In [37]:
score_summary = quiz_history.groupby('Word')[['Right Score', 'Wrong Score', 'Pinyin Wrong', 'Meaning Wrong']].sum().reset_index()
top_10_pinyin_wrong = score_summary.loc[score_summary['Pinyin Wrong']>1].sort_values(by='Pinyin Wrong', ascending=False)['Word'].to_list()[:10]
top_10_meaning_wrong = score_summary.loc[score_summary['Meaning Wrong']>1].sort_values(by='Meaning Wrong', ascending=False)['Word'].to_list()[:10]

In [38]:
quiz_history.groupby('Quiz Id')[['Right Score']].mean().reset_index()

,Quiz Id,Right Score
0,QW000001,0.200000
1,QW000002,0.800000
2,QW000003,0.600000
3,QW000004,0.600000
4,QW000005,0.700000
5,QW000006,0.600000
6,QW000007,0.800000
7,QW000008,0.500000
8,QW000009,0.300000
9,QW000010,0.572308
